In [1]:
import numpy as np
import tensorflow as tf
import InceptionV3_fine_tune2
import img_proc_scikit
from PIL import Image, ImageFile
from os import path, listdir, makedirs, fsdecode, getcwd
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


train_df = img_proc_scikit.TrainData('train.csv').make_train()
img_rows, img_cols = 299, 299  # Resolution of inputs
channel = 3
num_classes = 28
batch_size = 12

model = InceptionV3_fine_tune2.InceptionV3_make('InceptionV3_ft2.h5', 28)
f1_score = InceptionV3_fine_tune2.f1

train_dir = getcwd() + '\\train'
train_df['Preds'] = None
i = 0

In [ ]:
for line in train_df.values[:100]:
    sample = img_proc_scikit.save_rgb_image(line[0], line[2], train_dir, labeled_dir=None, save_img=False)
    sample = resize(sample, (299, 299))
    sample = np.array(sample)
    line[3] = model.predict(np.expand_dims(sample, axis=0), batch_size=1)[0]
    i += 1
    if (i % 100) == 0:
        print('{} images were predicted'.format(i))

C:\Users\User\Anaconda2\envs\ML_env\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [80]:
train_df.to_csv('predicted.csv')

In [4]:
import pickle
with open('train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)


In [5]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [11]:
Estimators = [RFC(n_estimators=50, max_depth=25) for x in range(28)]

In [12]:
targets = np.array(list(map(np.array,train_df['Cat'].values)))
X = np.array(list(map(np.array,train_df['Preds'].values)))
for column, estimator in enumerate(Estimators):
    y_pred = targets[:,column]
    estimator.fit(X, y_pred) 
    print('Esatimator no. {} was trained'.format(column))    

Esatimator no. 0 was trained
Esatimator no. 1 was trained
Esatimator no. 2 was trained
Esatimator no. 3 was trained
Esatimator no. 4 was trained
Esatimator no. 5 was trained
Esatimator no. 6 was trained
Esatimator no. 7 was trained
Esatimator no. 8 was trained
Esatimator no. 9 was trained
Esatimator no. 10 was trained
Esatimator no. 11 was trained
Esatimator no. 12 was trained
Esatimator no. 13 was trained
Esatimator no. 14 was trained
Esatimator no. 15 was trained
Esatimator no. 16 was trained
Esatimator no. 17 was trained
Esatimator no. 18 was trained
Esatimator no. 19 was trained
Esatimator no. 20 was trained
Esatimator no. 21 was trained
Esatimator no. 22 was trained
Esatimator no. 23 was trained
Esatimator no. 24 was trained
Esatimator no. 25 was trained
Esatimator no. 26 was trained
Esatimator no. 27 was trained


In [13]:
train_df['Forest_preds'] = None

In [14]:
i = 0
for line in train_df.values[:100]:
    line[4] = [estimator.predict([line[3]])[0] for estimator in Estimators]
    i += 1
    if (i % 100) == 0:
        print('{} lines were predicted'.format(i))

100 lines were predicted


In [15]:
train_df['Forest_preds'][:5]

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Forest_preds, dtype: object

In [16]:
import pickle
# now you can save it to a file
with open('train_df.pkl', 'wb') as f:
    pickle.dump(train_df, f)

with open('Forests.pkl', 'wb') as f:
    pickle.dump(Estimators, f)

In [68]:
train_df['Forest_preds'][:5]

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Forest_preds, dtype: object

In [17]:
scorer = f1_score(np.array(list(map(np.array,train_df['Cat'].values[0:100]))), np.array(list(map(np.array,train_df['Forest_preds'].values[0:100]))))

In [18]:
import keras.backend as K
K.eval(scorer)

0.68883055

In [ ]:
#with open('filename.pkl', 'rb') as f:
#    clf = pickle.load(f)